# 1 Kafka

## 1.1 Api admin

Instalar librerías en python

``` 
pip install confluent_kafka
pip install avro
```



In [4]:
from confluent_kafka.admin import AdminClient, NewTopic


admin_client = AdminClient({
    "bootstrap.servers": "localhost:9092"
})

topic_list = []
topic_list.append(NewTopic("topic-example", 3, 1))
admin_client.create_topics(topic_list)


{'topic-example': <Future at 0x104374100 state=running>}

## 1.2 Api producer

In [5]:
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='localhost:9092')


In [6]:

producer.send('topic-example', b'example topic')
producer.send('topic-example', value=b'value_A', key=b'key')


# producer.send('topic-example', value='value_A', key='key', headers=None, partition=None, timestamp_ms=None)


producer.

In [ ]:
Ejercicio: implementar un productor con compresión gzip 

## 1.3 Api consumer

In [ ]:
from kafka import KafkaConsumer

consumer = KafkaConsumer('topic-example', group_id='consumer_a', bootstrap_servers='localhost:9092')

for msg in consumer:
    print (msg)
    break

## 1.4 Schema Registry

In [28]:
# Crea el schema con nombre person en el schema registry
{
  "type": "record",
  "name": "Person",
  "namespace": "com.curso.kafka",
  "fields": [
    {
      "name": "firstName",
      "type": "string"
    },
    {
      "name": "lastName",
      "type": "string"
    },
    {
      "name": "age",
      "type": "long"
    }
  ]
}

{'type': 'record',
 'name': 'Person',
 'namespace': 'com.curso.kafka',
 'fields': [{'name': 'firstName', 'type': 'string'},
  {'name': 'lastName', 'type': 'string'},
  {'name': 'age', 'type': 'long'}]}

In [34]:
import requests
schema_name = 'newSchema'

r = requests.get('http://localhost:8081/subjects/' + schema_name +'/versions/1')

sc = r.json()['schema']

from confluent_kafka.avro import AvroProducer
from confluent_kafka import avro
from confluent_kafka.schema_registry import (_MAGIC_BYTE,
                                             Schema,
                                             topic_subject_name_strategy)



schema = Schema(sc, schema_type="JSON")
schema.schema_str

'{"type":"record","name":"newSchema","namespace":"com.curso.kafka","fields":[{"name":"firstName","type":"string"},{"name":"lastName","type":"string"},{"name":"age","type":"long"}]}'

In [35]:

producer_config = {
        "bootstrap.servers": "localhost:9092",
        "schema.registry.url": "http://localhost:8081"
    }
producer = AvroProducer(producer_config, default_value_schema=schema)

/var/folders/71/197r1l414ysbpbjvy9ktvzcr0000gn/T/ipykernel_1510/412467054.py:5: DeprecationWarning: AvroProducer has been deprecated. Use AvroSerializer instead.
  producer = AvroProducer(producer_config, default_value_schema=schema)


In [37]:
value = {'firstName': 'Pepe', 'lastName': 'Garcia', 'age': 30}

producer.produce(topic='people', value=value)

ClientError: Invalid Avro schema:422 message:{'error_code': 42201, 'message': 'Invalid schema <confluent_kafka.schema_registry.schema_registry_client.Schema object at 0x124805300> with refs [] of type AVRO, details: com.fasterxml.jackson.core.JsonParseException: Unexpected character (\'<\' (code 60)): expected a valid value (JSON String, Number, Array, Object or token \'null\', \'true\' or \'false\')\n at [Source: (String)"<confluent_kafka.schema_registry.schema_registry_client.Schema object at 0x124805300>"; line: 1, column: 2]'}